# Finding a Consensus Sequence

This code is implementation of Chapter 2 from Bioinformatics Algorithms (Pevzner, Compeau).

In [17]:
import pandas as pd
from collections import Counter
   
seqs = ['TCGGGGGTTTTT', 
        'CCGGTGACTTAC', 
        'ACGGGGATTTTC', 
        'TTGGGGACTTTT',
        'AAGGGGACTTCC',
        'TTGGGGACTTCC',
        'TCGGGGATTCAT',
        'TCGGGGATTCCT',
        'TAGGGGACCTAC',
        'TCGGGTATAACC']
data = {}
for i, seq in enumerate(seqs):
    data[i+1] = list(seq)

df = pd.DataFrame(data).T
df

,0,1,2,3,4,5,6,7,8,9,10,11
1,T,C,G,G,G,G,G,T,T,T,T,T
2,C,C,G,G,T,G,A,C,T,T,A,C
3,A,C,G,G,G,G,A,T,T,T,T,C
4,T,T,G,G,G,G,A,C,T,T,T,T
5,A,A,G,G,G,G,A,C,T,T,C,C
6,T,T,G,G,G,G,A,C,T,T,C,C
7,T,C,G,G,G,G,A,T,T,C,A,T
8,T,C,G,G,G,G,A,T,T,C,C,T
9,T,A,G,G,G,G,A,C,C,T,A,C
10,T,C,G,G,G,T,A,T,A,A,C,C


Now we have a DataFrame where we can get all the bases in each position to find the base(s) occurring most frequently.  We'll create a second DataFrame where the less frequent bases are put into lowercase.  We'll also create a profile that gives us the frequency as a percentage.  Note that we can create DataFrame of zeros easily, but we need the percentage profile to be initialized with "0." to indicate that we will store floats not integers.

Lastly, we'll create a "simple consensus" that simply uses the singly most frequently occurring base.  When there is a tie, a random base is chosen.

In [11]:
freq_df = pd.DataFrame()
nucs = list('ACGT')
num_seqs = len(seqs)
counts_df = pd.DataFrame(0, index=nucs, columns=list(range(num_seqs)))
profile_df = pd.DataFrame(0., index=nucs, columns=list(range(num_seqs)))
simple_consensus = []

for col in df.columns:
    bases = df[col].values.tolist()
    base_count = Counter(bases)
    most_common = max(set(bases), key=bases.count)
    simple_consensus.append(most_common)
    
    freq_df[col] = list(map(lambda b: b if b == most_common else b.lower(), bases))
    
    for nuc in nucs:
        counts_df.at[nuc, col] = base_count[nuc]
        profile_df.at[nuc, col] = base_count[nuc] / len(seqs)

print('Consensus')
print(''.join(simple_consensus))

print('Frequency')
freq_df

Consensus
TCGGGGATTTCC
Frequency


,0,1,2,3,4,5,6,7,8,9,10,11
0,T,C,G,G,G,G,g,T,T,T,t,t
1,c,C,G,G,t,G,A,c,T,T,a,C
2,a,C,G,G,G,G,A,T,T,T,t,C
3,T,t,G,G,G,G,A,c,T,T,t,t
4,a,a,G,G,G,G,A,c,T,T,C,C
5,T,t,G,G,G,G,A,c,T,T,C,C
6,T,C,G,G,G,G,A,T,T,c,a,t
7,T,C,G,G,G,G,A,T,T,c,C,t
8,T,a,G,G,G,G,A,c,c,T,a,C
9,T,C,G,G,G,t,A,T,a,a,C,C


In [19]:
print('Counts')
counts_df

Counts


,0,1,2,3,4,5,6,7,8,9,10,11
A,2,2,0,0,0,0,9,0,1,1,3.0,0.0
C,1,6,0,0,0,0,0,5,1,2,4.0,6.0
G,0,0,10,10,9,9,1,0,0,0,0.0,0.0
T,7,2,0,0,1,1,0,5,8,7,3.0,4.0


In [13]:
print('Profile')
profile_df

Profile


,0,1,2,3,4,5,6,7,8,9,10,11
A,0.2,0.2,0.0,0.0,0.0,0.0,0.9,0.0,0.1,0.1,0.3,0.0
C,0.1,0.6,0.0,0.0,0.0,0.0,0.0,0.5,0.1,0.2,0.4,0.6
G,0.0,0.0,1.0,1.0,0.9,0.9,0.1,0.0,0.0,0.0,0.0,0.0
T,0.7,0.2,0.0,0.0,0.1,0.1,0.0,0.5,0.8,0.7,0.3,0.4


In [14]:
scores = []
for col in freq_df:
    bases = freq_df[col].values.tolist()
    lower = list(filter(lambda b: b.islower(), bases))
    scores.append(len(lower))
    
print(scores)
print(sum(scores))

[3, 4, 0, 0, 1, 1, 1, 5, 2, 3, 6, 4]
30


For the "real" consensus, we'll take into account all bases occurring at a frequency of 40% or greater.

In [15]:
consensus = []
for col in profile_df.columns:
    profile = profile_df[col]
    max_bases = profile[profile >= .4].index.tolist()
    consensus.append('/'.join(max_bases))
    
print('Consensus')
print('  '.join(consensus))

Consensus
T  C  G  G  G  G  A  C/T  T  T  C  C/T
